In [ ]:
from arcgis.gis import GIS
from arcgis import features
import pandas as pd
import os
import datetime as dt
import shutil

In [ ]:
# login to arcgis online
gis = GIS('https://arcgis.com', 'Emeka_Orji')

Enter password: ········


In [ ]:
'''
import pyodbc
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DESKTOP-9GUC0GM;'
                      'Database=covid19;'
                      'Trusted_Connection=yes;')

cur = conn.cursor()

sql_cal = pd.read_sql_query('SELECT * FROM covid19.dbo.covid19_call_records', conn)
print(type(sql_cal))

cur.close()

'''

"\nimport pyodbc \nconn = pyodbc.connect('Driver={SQL Server};'\n                      'Server=DESKTOP-9GUC0GM;'\n                      'Database=covid19;'\n                      'Trusted_Connection=yes;')\n\ncur = conn.cursor()\n\nsql_cal = pd.read_sql_query('SELECT * FROM covid19.dbo.covid19_call_records', conn)\nprint(type(sql_cal))\n\ncur.close()\n\n"

In [ ]:
# read in the data from database... here from csv
csv1 = os.path.join('mgic_update', 'testing_append1.csv')
calls_1 = pd.read_csv(csv1)
calls_1.head()

,state_id,state,latitude,longitude,june_1,june_2,june_3
0,1,Abia,5.452735,7.524841,10,18,14
1,2,Adamawa,9.326475,12.398385,26,14,10
2,3,Akwa Ibom,4.905737,7.853668,18,14,10
3,4,Anambra,6.220900,6.936956,23,17,13
4,5,Bauchi,10.776062,9.999194,18,15,11


In [ ]:
calls_1.shape

(37, 7)

In [ ]:
now_time = str(int(dt.datetime.now().timestamp()))

In [ ]:
item_prop = {'title':'states calls database' + now_time}
csv_item = gis.content.add(item_properties=item_prop, data=csv1)
csv_item

<Item title:"states calls database1594902232" type:CSV owner:Emeka_Orji>

In [ ]:
calls_item = csv_item.publish()
calls_item

<Item title:"states calls database1594902232" type:Feature Layer Collection owner:Emeka_Orji>

In [ ]:
### Apply new call update from database
# read new data into python dataframe

csv2 = os.path.join('mgic_update', 'june_1_3.csv')
calls_2 = pd.read_csv(csv2)
calls_2.head()

,state_id,state,latitude,longitude,june_1,june_2,june_3
0,1,Abia,5.452735,7.524841,13,20,20
1,2,Adamawa,9.326475,12.398385,29,16,16
2,3,Akwa Ibom,4.905737,7.853668,21,16,16
3,4,Anambra,6.220900,6.936956,26,19,19
4,5,Bauchi,10.776062,9.999194,25,17,17


In [ ]:
calls_2.shape

(37, 7)

In [ ]:
calls_flayer = calls_item.layers[0]
calls_fset = calls_flayer.query()

In [ ]:
overlap_rows = pd.merge(left = calls_fset.sdf, right = calls_2,
                       how = 'inner', on = 'state_id')

In [ ]:
features_for_update = []
all_features = calls_fset.features
all_features[-1]

{"geometry": {"x": 692805.968018234, "y": 1359616.4982888899, "spatialReference": {"wkid": 102100, "latestWkid": 3857}}, "attributes": {"state_id": 37, "state": "Zamfara", "latitude": 12.1221805, "longitude": 6.2235819, "june_1": 5, "june_2": 12, "june_3": 6, "ObjectId": 37, "SHAPE": {"x": 692805.968018234, "y": 1359616.4982888899, "spatialReference": {"wkid": 102100, "latestWkid": 3857}}}}

In [ ]:
from arcgis import geometry
from copy import deepcopy

In [ ]:
for state_id in overlap_rows['state_id']:
    original_feature = [f for f in all_features if f.attributes['state_id'] == state_id][0]
    feature_to_be_updated = deepcopy(original_feature)
    matching_row = calls_2.where(calls_2.state_id == state_id).dropna()
    input_geometry = {'y':float(matching_row['latitude']),
                     'x':float(matching_row['longitude'])}
    output_geometry = geometry.project(geometries = [input_geometry],
                                      in_sr = 4326,
                                      out_sr =
                calls_fset.spatial_reference['latestWkid'], gis=gis)

    feature_to_be_updated.geometry = output_geometry[0]
    feature_to_be_updated.attributes['longitude'] = float(
    matching_row['longitude'])
    feature_to_be_updated.attributes['state_id'] = int(matching_row['state_id'])
    feature_to_be_updated.attributes['state'] = matching_row['state'].values[0]
    feature_to_be_updated.attributes['latitude'] = float(matching_row['latitude'])
    feature_to_be_updated.attributes['june_1'] = int(matching_row['june_1'])
    feature_to_be_updated.attributes['june_2'] = int(matching_row['june_2'])
    feature_to_be_updated.attributes['june_3'] = int(matching_row['june_3'])

    features_for_update.append(feature_to_be_updated)


In [ ]:
features_for_update[:3]

[{"geometry": {"x": 837661.512948144, "y": 607914.0666610777}, "attributes": {"state_id": 1, "state": "Abia", "latitude": 5.452735400000001, "longitude": 7.524841400000001, "june_1": 13, "june_2": 20, "june_3": 20, "ObjectId": 1, "SHAPE": {"x": 837661.5129481438, "y": 607914.0666610777, "spatialReference": {"wkid": 102100, "latestWkid": 3857}}}},
 {"geometry": {"x": 1380181.9382548085, "y": 1042833.9385164682}, "attributes": {"state_id": 2, "state": "Adamawa", "latitude": 9.3264751, "longitude": 12.398385300000001, "june_1": 29, "june_2": 16, "june_3": 16, "ObjectId": 2, "SHAPE": {"x": 1380181.9382548083, "y": 1042833.9385164682, "spatialReference": {"wkid": 102100, "latestWkid": 3857}}}},
 {"geometry": {"x": 874266.266959682, "y": 546772.6297957378}, "attributes": {"state_id": 3, "state": "Akwa Ibom", "latitude": 4.9057371, "longitude": 7.8536675, "june_1": 21, "june_2": 16, "june_3": 16, "ObjectId": 3, "SHAPE": {"x": 874266.266959682, "y": 546772.6297957378, "spatialReference": {"wki

In [ ]:
calls_flayer.edit_features(updates = features_for_update)

{'addResults': [],
 'updateResults': [{'objectId': 1,
   'uniqueId': 1,
   'globalId': None,
   'success': True},
  {'objectId': 2, 'uniqueId': 2, 'globalId': None, 'success': True},
  {'objectId': 3, 'uniqueId': 3, 'globalId': None, 'success': True},
  {'objectId': 4, 'uniqueId': 4, 'globalId': None, 'success': True},
  {'objectId': 5, 'uniqueId': 5, 'globalId': None, 'success': True},
  {'objectId': 6, 'uniqueId': 6, 'globalId': None, 'success': True},
  {'objectId': 7, 'uniqueId': 7, 'globalId': None, 'success': True},
  {'objectId': 8, 'uniqueId': 8, 'globalId': None, 'success': True},
  {'objectId': 9, 'uniqueId': 9, 'globalId': None, 'success': True},
  {'objectId': 10, 'uniqueId': 10, 'globalId': None, 'success': True},
  {'objectId': 11, 'uniqueId': 11, 'globalId': None, 'success': True},
  {'objectId': 12, 'uniqueId': 12, 'globalId': None, 'success': True},
  {'objectId': 13, 'uniqueId': 13, 'globalId': None, 'success': True},
  {'objectId': 14, 'uniqueId': 14, 'globalId': Non

In [ ]:
csv3 = os.path.join('mgic_update', 'june_4_8.csv')
calls_3 = pd.read_csv(csv3)
print(calls_3.shape)
calls_3.head()

(37, 7)


,state_id,state,june_4,june_5,june_6,june_7,june_8
0,1,Abia,11,8,8,17,10
1,2,Adamawa,14,9,6,7,5
2,3,Akwa Ibom,13,11,10,31,64
3,4,Anambra,27,20,8,18,15
4,5,Bauchi,16,23,29,13,10


In [ ]:
# get fields of call feature layer
calls_fields = calls_flayer.manager.properties.fields
calls_fields[3]

{
  "name": "longitude",
  "type": "esriFieldTypeDouble",
  "actualType": "float",
  "alias": "longitude",
  "sqlType": "sqlTypeFloat",
  "nullable": true,
  "editable": true,
  "visible": true,
  "domain": null,
  "defaultValue": null
}

In [ ]:
for field in calls_fields:
    print(f'{field.name:13}| {field.alias:13}| {field.type:25}| {field.sqlType}')

state_id     | state_id     | esriFieldTypeInteger     | sqlTypeInteger
state        | state        | esriFieldTypeString      | sqlTypeNVarchar
latitude     | latitude     | esriFieldTypeDouble      | sqlTypeFloat
longitude    | longitude    | esriFieldTypeDouble      | sqlTypeFloat
june_1       | june_1       | esriFieldTypeInteger     | sqlTypeInteger
june_2       | june_2       | esriFieldTypeInteger     | sqlTypeInteger
june_3       | june_3       | esriFieldTypeInteger     | sqlTypeInteger
ObjectId     | ObjectId     | esriFieldTypeOID         | sqlTypeInteger


In [ ]:
template_field = dict(deepcopy(calls_fields[0]))
template_field

{'name': 'state_id',
 'type': 'esriFieldTypeInteger',
 'actualType': 'int',
 'alias': 'state_id',
 'sqlType': 'sqlTypeInteger',
 'nullable': True,
 'editable': True,
 'visible': True,
 'domain': None,
 'defaultValue': None}

In [ ]:
new_field_names = list(calls_3.columns.difference(calls_1.columns))
new_field_names

['june_4', 'june_5', 'june_6', 'june_7', 'june_8']

In [ ]:
fields_to_add = []
for new_field_name in new_field_names:
    current_field = deepcopy(template_field)
    current_field['name'] = new_field_name.lower()
    current_field['alias'] = new_field_name

    fields_to_add.append(current_field)

len(fields_to_add)

5

In [ ]:
fields_to_add[3]

{'name': 'june_7',
 'type': 'esriFieldTypeInteger',
 'actualType': 'int',
 'alias': 'june_7',
 'sqlType': 'sqlTypeInteger',
 'nullable': True,
 'editable': True,
 'visible': True,
 'domain': None,
 'defaultValue': None}

In [ ]:
calls_flayer.manager.add_to_definition({'fields':fields_to_add})

{'success': True}

In [ ]:
new_calls_field = calls_flayer.manager.properties.fields
len(new_calls_field)

13

In [ ]:
for field in new_calls_field:
    print(f'{field.name:10} | {field.type}')

state_id   | esriFieldTypeInteger
state      | esriFieldTypeString
latitude   | esriFieldTypeDouble
longitude  | esriFieldTypeDouble
june_1     | esriFieldTypeInteger
june_2     | esriFieldTypeInteger
june_3     | esriFieldTypeInteger
ObjectId   | esriFieldTypeOID
june_4     | esriFieldTypeInteger
june_5     | esriFieldTypeInteger
june_6     | esriFieldTypeInteger
june_7     | esriFieldTypeInteger
june_8     | esriFieldTypeInteger


In [ ]:
calls_fset2 = calls_flayer.query()
calls_features2 = calls_fset2.features

In [ ]:
features_for_update = []
for state_id in calls_3['state_id']:
    matching_row = calls_3.where(calls_3.state_id == state_id).dropna()

    original_feature = [f for f in calls_features2 if
                        f.attributes['state_id'] == state_id][0]
    feature_to_update = deepcopy(original_feature)

    feature_to_update.attributes['june_4'] = int(matching_row['june_4'])
    feature_to_update.attributes['june_5'] = int(matching_row['june_5'])
    feature_to_update.attributes['june_6'] = int(matching_row['june_6'])
    feature_to_update.attributes['june_7'] = int(matching_row['june_7'])
    feature_to_update.attributes['june_8'] = int(matching_row['june_8'])

    features_for_update.append(feature_to_update)

In [ ]:
features_for_update[-1]

{"geometry": {"x": 692805.968018234, "y": 1359616.4982888887}, "attributes": {"state_id": 37, "state": "Zamfara", "latitude": 12.1221805, "longitude": 6.2235819, "june_1": 5, "june_2": 12, "june_3": 12, "ObjectId": 37, "june_4": 8, "june_5": 2, "june_6": 2, "june_7": 4, "june_8": 4}}

In [ ]:
calls_flayer.edit_features(updates = features_for_update)

{'addResults': [],
 'updateResults': [{'objectId': 1,
   'uniqueId': 1,
   'globalId': None,
   'success': True},
  {'objectId': 2, 'uniqueId': 2, 'globalId': None, 'success': True},
  {'objectId': 3, 'uniqueId': 3, 'globalId': None, 'success': True},
  {'objectId': 4, 'uniqueId': 4, 'globalId': None, 'success': True},
  {'objectId': 5, 'uniqueId': 5, 'globalId': None, 'success': True},
  {'objectId': 6, 'uniqueId': 6, 'globalId': None, 'success': True},
  {'objectId': 7, 'uniqueId': 7, 'globalId': None, 'success': True},
  {'objectId': 8, 'uniqueId': 8, 'globalId': None, 'success': True},
  {'objectId': 9, 'uniqueId': 9, 'globalId': None, 'success': True},
  {'objectId': 10, 'uniqueId': 10, 'globalId': None, 'success': True},
  {'objectId': 11, 'uniqueId': 11, 'globalId': None, 'success': True},
  {'objectId': 12, 'uniqueId': 12, 'globalId': None, 'success': True},
  {'objectId': 13, 'uniqueId': 13, 'globalId': None, 'success': True},
  {'objectId': 14, 'uniqueId': 14, 'globalId': Non